In [1]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.impute import SimpleImputer
import Featurizer as ft

df = pd.read_csv('../data/train.csv')

In [2]:
a = ft.main(df)

In [3]:
a

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,palabra_renta,palabra_super,palabra_lago,palabra_bosque,palabra_avenida,palabra_hospital,palabra_pileta,palabra_solarium,palabra_gas,ciudad-provincia
0,254099.0,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,8.116114,2.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"Benito Juárez, Distrito Federal"
1,53461.0,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"La Magdalena Contreras, Distrito Federal"
2,247984.0,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.000000,3.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"Tonalá, Jalisco"
3,209067.0,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.000000,2.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"Zinacantepec, Edo. de México"
4,185997.0,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.000000,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Zapopan, Jalisco"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879.0,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.000000,2.0,2.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,"Zinacantepec, Edo. de México"
239996,259178.0,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.000000,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Toluca, Edo. de México"
239997,131932.0,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.000000,2.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"Benito Juárez, Distrito Federal"
239998,146867.0,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.000000,4.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,"Iztapalapa, Distrito Federal"


In [4]:
a = ft.OHE(df,["tipodepropiedad","provincia"])

In [5]:
a

,id,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,lat,lng,...,provincia_24,provincia_25,provincia_26,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,provincia_33
0,254099.0,8.116114,2.0,1.0,2.0,80.0,80.000000,23533.0,20.696818,-99.509221,...,0,0,0,0,0,0,0,0,0,0
1,53461.0,10.000000,3.0,2.0,2.0,268.0,180.000000,24514.0,19.310205,-99.227655,...,0,0,0,0,0,0,0,0,0,0
2,247984.0,5.000000,3.0,2.0,2.0,144.0,166.000000,48551.0,20.696818,-99.509221,...,0,0,0,0,0,0,0,0,0,0
3,209067.0,1.000000,2.0,1.0,1.0,63.0,67.000000,53666.0,19.301890,-99.688015,...,0,0,0,0,0,0,0,0,0,0
4,185997.0,10.000000,2.0,1.0,1.0,95.0,95.000000,47835.0,20.696818,-99.509221,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879.0,0.000000,2.0,2.0,1.0,67.0,176.765145,53666.0,20.696818,-99.509221,...,0,0,0,0,0,0,0,0,0,0
239996,259178.0,0.000000,3.0,3.0,3.0,200.0,250.000000,51954.0,19.294665,-99.692916,...,0,0,0,0,0,0,0,0,0,0
239997,131932.0,20.000000,2.0,1.0,2.0,138.0,138.000000,50003995.0,20.696818,-99.509221,...,0,0,0,0,0,0,0,0,0,0
239998,146867.0,20.000000,4.0,0.0,4.0,235.0,137.000000,24162.0,19.366651,-99.082246,...,0,0,0,0,0,0,0,0,0,0


In [2]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False

contiene_frase=np.vectorize(contiene_frase)

In [3]:
def cantidad_de_public(data,colum,palabra):
    return data[contiene_frase(data[colum],palabra)].dia.count()

def serie_que_contiene_palabra(data,colum,palabra):
        return contiene_frase(data[colum],palabra)

# Completo los datos faltantes

In [4]:
imp = SimpleImputer(strategy="constant")
df["provincia"] = imp.fit_transform(df[["provincia"]])
df["ciudad"] = imp.fit_transform(df[["ciudad"]])

In [5]:
df['ciudad-provincia'] = df[['ciudad', 'provincia']].apply(lambda x: ', '.join(x), axis=1)

In [6]:
df = df.reset_index()

# Busco en las descripciones

In [7]:
busqueda = "descripcion"
df1 = df.copy()
df1[busqueda] = df1[busqueda].fillna(value="-")
df1.shape

(60000, 25)

In [8]:
palabra = "excelente"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

17445

In [9]:
palabra = "centro"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

6419

In [10]:
palabra = "patio"
palabra2 = "jardin"
palabra3 = "jardín"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

25379

In [11]:
palabra = "hermosa"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

5443

In [12]:
palabra = "grande"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

2871

In [13]:
palabra = "universidad"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

987

In [14]:
palabra = "ilum"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)


5236

In [15]:
palabra = "seguridad"
palabra2 = "vigilancia"
palabra3 = "proteccion"
palabra4 = "protecciones"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)|serie_que_contiene_palabra(df1,busqueda,palabra4)
df1[palabra].value_counts()[1]

14641

In [16]:
palabra = "tienda"
palabra2 = "comercial"
palabra3 = "local"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

9880

In [17]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

31961

In [18]:
palabra = "lujo"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

3477

In [19]:
palabra = "servicio"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

21521

In [20]:
palabra = "remodelada"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

1611

In [21]:
palabra = "autos"
palabra2 = "auto"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

19584

In [22]:
palabra = "completo"
palabra2 = "completos"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

16745

In [23]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

31961

In [24]:
palabra = "cerca de"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

7440

In [25]:
palabra = "con vista"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

2315

# Busco en las direcciones

In [26]:
busqueda = "direccion"

df1[busqueda] = df1[busqueda].fillna(value="-")

In [27]:
palabra = "avenida"
palabra2 = "av"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

5044

In [28]:
df1["largo descripcion"] = df1.descripcion.transform(lambda x: len(x))

# Creo el CSV

In [29]:
df1.to_csv("../data/features.csv", index=False)